<div style = 'background-color: #6482AD; text-align:center; padding: 10px; color:#F5EDED'>
    <h1>Support Vector Regression</h1>
    <h2><a href = 'https://www.linkedin.com/in/ahmedsharaf9/' style = "text-decoration: none; color: #F5EDED">Ahmed Sharaf</a></h2>
</div>   </ul>
</div>

<div style = 'background-color: #6482AD; text-align:center; padding: 10px; color:#F5EDED'>
    <h2>Import Libraries</h2>
</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

<div style = 'background-color: #6482AD; text-align:center; padding: 10px; color:#F5EDED'>
    <h2>Import Data</h2>
</div>

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


<div style = 'background-color: #6482AD; text-align:center; padding: 10px; color:#F5EDED'>
    <h2>EDA And Data Preprocessing</h2>
</div>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
for col in df.columns:
    if df[col].dtypes == 'category':
        print(df[col].value_counts())
        print('\n','#' * 10)


sex
Male      157
Female     87
Name: count, dtype: int64

 ##########
smoker
No     151
Yes     93
Name: count, dtype: int64

 ##########
day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

 ##########
time
Dinner    176
Lunch      68
Name: count, dtype: int64

 ##########


#### From Here We Apply Label Encoding in [sex, smoker, time]
#### And Apply OneHot Encoding in [day]

In [5]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [6]:
# first we split data to train, test to avoid dataleakage
from sklearn.model_selection import train_test_split
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=10)

In [7]:
# lets apply labelencoding
from sklearn.preprocessing import LabelEncoder
lbl_encoders = []
for col in ['sex', 'smoker', 'time']:
    enc = LabelEncoder()
    X_train[col] = enc.fit_transform(X_train[col])
    X_test[col] = enc.transform(X_test[col])
    # add encoder to list
    lbl_encoders.append({col:enc})

### Some Note To Syntax 
### transformers -> Take List Of Tuple -> [('name', object tranform, [index of column that apply transform])]
### remainder -> Two Values 
###                  'drop' -> drop all columns in original data whithout drop the columns after transform 
###                  'passthrough' -> don't drop any columns 
### in onehot encoding -> drop = 'first'      means after apply encoding drop fisrt column because we can represent it using remaining columns

In [8]:
# lets apply OneHot Encoder
# to apply one hot encoding we need to use columntransformer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('ohe', OneHotEncoder(drop='first'), [3])], remainder = 'passthrough')

X_train = ct.fit_transform(X_train)

In [9]:
X_test = ct.transform(X_test)

<div style = 'background-color: #6482AD; text-align:center; padding: 10px; color:#F5EDED'>
    <h2>Build SVR Model</h2>
</div>

In [10]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)

# lets get metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print('Error Metrcis : ')
print(f'MSE = {mean_squared_error(y_test, y_pred)}')
print(f'RMSE = {np.sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAE = {mean_absolute_error(y_test, y_pred)}')
print()
print('Accuracy Metric : ')
print(f'R2 = {r2_score(y_test, y_pred)}')

Error Metrcis : 
MSE = 45.69213615168339
RMSE = 6.759595857126622
MAE = 4.1486423210190235

Accuracy Metric : 
R2 = 0.4602811456115927


<div style = 'background-color: #6482AD; text-align:center; padding: 10px; color:#F5EDED'>
    <h2>Build SVR Model With HyperParamters Tuning</h2>
</div>

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
}
grid_model = GridSearchCV(SVR(), param_grid=parameters, cv = 5)
grid_model.fit(X_train, y_train)
y_pred = grid_model.predict(X_test)
# lets get metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print('Error Metrcis : ')
print(f'MSE = {mean_squared_error(y_test, y_pred)}')
print(f'RMSE = {np.sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAE = {mean_absolute_error(y_test, y_pred)}')
print()
print('Accuracy Metric : ')
print(f'R2 = {r2_score(y_test, y_pred)}')

Error Metrcis : 
MSE = 41.63859886764171
RMSE = 6.452797755054912
MAE = 3.8685147526100234

Accuracy Metric : 
R2 = 0.5081618245078687


In [12]:
grid_model.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}